Task 4 :Bengali
Transliterate to english and using char TF-IDF with SVM and Ensemble model

Train dataset(splitting)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install emot --upgrade
# to convert emojis to text
!pip install emoji
#to expand a contracted words
!pip install demoji
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 752.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 892.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.6 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
import emot
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import emoji
import demoji
import contractions
import nltk
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option("max_colwidth" ,220)
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stopwords = stopwords.words("english")
with open('/content/drive/MyDrive/Hasoc 2023/Task 4/Bengali/bn.txt', encoding = 'utf-8') as f:
    ben_stopwords = f.readlines()
    for i in range(len(ben_stopwords)):
        ben_stopwords[i] = re.sub('\n','', ben_stopwords[i])
stopwords = english_stopwords + ben_stopwords
#english_stemmer = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-5-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
punct="!#$%&\'()*+,-.’’/:;<=>?@[\\]^_`{|}~’“‘"

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', str(text))

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))#to remove @ and its following word
    text = contractions.fix(text, slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation]) #to remove punctuation
    text="".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#to remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)
    return text

In [ ]:
import pandas as pd


In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/Hasoc 2023/Task 4/Bengali/train_BE_AH_HASOC2023.csv')
df2=pd.read_csv('/content/drive/MyDrive/Hasoc 2023/Task 4/Bengali/test_BE_AH_HASOC2023.csv')

In [ ]:
df1['clean_text'] = df1['text'].apply(lambda x:emo(x))
df1['clean_text'] = df1['clean_text'].apply(lambda x:remove_urls(x))
df1['clean_text'] = df1['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
df2['clean_text'] = df2['text'].apply(lambda x:emo(x))
df2['clean_text'] = df2['clean_text'].apply(lambda x:remove_urls(x))
df2['clean_text'] = df2['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
df1['clean_text']

0                                                                                         শুনলাম মমতা ব্যানার্জি কোটা পদ্ধতি তুলে দিয়েছে তাহলে ব্রাত্য বসুর মুখে কোটা এটা কথার কথা তাহলে কোটা নিয়ে প্রাক্তন মুখ্যমন্ত্রীকে কটাক্ষ
1       প্রত্যেক দলের প্রত্যেক মন্ত্রী এমএলএ এমপি এদের একটা নিজস্ব কোটা কোটায় তারা যাকে খুশি চাকরি দিতে পারেন এবং চিরকাল এনারা সবাই এভাবে চাকরি দিয়েও এসেছেন। বরাবরই সংবিধানে আজও আছে। এবং স্বাভাবিকভাবে যেই দলের এমপি এমএলএ ত...
2       বিশ্ব জুড়ে বুদ্ধিজীবিদের সম্পর্ক থাকে বিশ্ববিদ্যালয়সেমিনার পাবলিশিং হাউস মিডিয়ার। বাংলায় দেখছি গরু চোরাকারবারিদের সাথে বুদ্ধিজীবীরা সম্পর্ক গড়ে তোলেছে।ভয় পাচ্ছি এরা বসে অর্থনীতির দিগন্ত খুলে দিয়েছে গরু চোরেরা নো...
3       বছর শত শত মানুষ হত্যা করছে সীমান্তে গণতান্ত্রিক আইনের শাসনের দেশ ভারতের সীমান্তরক্ষীরা। পৃথিবীর কোন সভ্য দেশের সীমান্তে মানুষ মানুষকে গুলি হত্যা । ইউরোপের অষ্ট্রিয়া পর্তুগাল পর্যন্ত গাড়ি চালিয়ে যেতে পারেন কেও আপনাক...
4                                                                                       

In [ ]:
df2['clean_text']

0                                                                                                                                         তাতে তোর কী চুলকুনি নারে সালা
1                                                                                                                                                          চটি চাটা হবে
2      kkr প্রসঙ্গ তুললে শাহরুখের ছবি দেয়া বাধ্যতামূলক শাহরুখ kkr সহমালিক পারে ক্রিকেটার নয়। ক্রিকেট এবং ক্রিকেটারকে ভালবেসে আপামর জনতা খেলা দেখে দলের মালিককে দেখে নয়।
3                                                                                        হ্যা হ্যা ভিখিরি বলতেই পাশের দেশের লোকের লাগছে। যার নাম ধরে ডাকলেই চিনতে পারে।
4                                                       এটা নিজের মেয়ের নাম রাখবেন নাম ধরে ডাকবেন beaming face smiling eyes grinning face sweat grinning squinting face
                                                                                     ...                                                                        

Transliteration

In [ ]:
!git clone https://github.com/libindic/Transliteration.git

Cloning into 'Transliteration'...
remote: Enumerating objects: 354, done.
remote: Total 354 (delta 0), reused 0 (delta 0), pack-reused 354
Receiving objects: 100% (354/354), 908.74 KiB | 10.82 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [ ]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
!python -m pip install --user virtualenv

  Obtaining dependency information for virtualenv from https://files.pythonhosted.org/packages/17/8d/6989e5dcd812520cbf9f31be2b08643ae3a895586601bbab501df8ed6e54/virtualenv-20.24.3-py3-none-any.whl.metadata
  Obtaining dependency information for distlib<1,>=0.3.7 from https://files.pythonhosted.org/packages/43/a0/9ba967fdbd55293bacfc1507f58e316f740a3b231fc00e3d86dc39bc185a/distlib-0.3.7-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 36.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!python -m virtualenv env

created virtual environment CPython3.10.12.final.0-64 in 1467ms
  creator CPython3Posix(dest=/content/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.2.1, setuptools==68.0.0, wheel==0.41.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
!source env/bin/activate

In [ ]:
pwd

'/content'

In [ ]:
!pip install -r '/content/Transliteration/requirements.txt'

  Cloning https://github.com/libindic/normalizer.git to /tmp/pip-install-aq_kqtbh/libindic-normalizer_8e95536530e548d782981c56920c25a9
  Running command git clone --filter=blob:none --quiet https://github.com/libindic/normalizer.git /tmp/pip-install-aq_kqtbh/libindic-normalizer_8e95536530e548d782981c56920c25a9
  Resolved https://github.com/libindic/normalizer.git to commit 88447f611e44ddfc9e9c6696650671afc10c25ae
  Preparing metadata (setup.py) ... done
Ignoring repoze.lru: markers 'python_version < "3"' don't match your environment
  Preparing metadata (setup.py) ... done
  Created wheel for libindic-utils: filename=libindic_utils-1.0.3-py3-none-any.whl size=9686 sha256=86c5a31021390844af488d1ef7de03c76cc978f12e18487c26001ca7fb10efce
  Stored in directory: /root/.cache/pip/wheels/a9/50/5f/64d267b1b00db903376596b944d6d7ca80ad721254d6eff902
  Created wheel for libindic_normalizer: filename=libindic_normalizer-0.0.0-py2.py3-none-any.whl size=6359 sha256=9b1d4a89facd39dfa8facea3edf9304f7f

In [ ]:
!pip install  '/content/Transliteration/.'

Processing ./Transliteration
  Preparing metadata (setup.py) ... done
  Created wheel for libindic.transliteration: filename=libindic.transliteration-0.4.1-py3-none-any.whl size=851872 sha256=a8fb3ae5f80014d174f3eeda2b24a4875fa7ba805f6288680e58825f10839831
  Stored in directory: /tmp/pip-ephem-wheel-cache-ktnn9s5g/wheels/94/ea/8b/a1558e8a38a7980bb506204694fd26148d79d71167f025eb79
Successfully built libindic.transliteration


In [ ]:
from libindic.transliteration import getInstance


In [ ]:
ldata=df1.clean_text
ldata=ldata.tolist()

In [ ]:
len(ldata)

1281

In [ ]:
fin1=[]
for i in ldata:
  t = getInstance()
  t_text = t.transliterate(str(i), "en_IN")
  fin1.append(t_text)
len(fin1)

1281

In [ ]:
df=pd.DataFrame(fin1)
df=df.rename(columns={0:'texttranslit'})
df.head(20)

,texttranslit
0,sunalaam mamathaa byaanaarji kootaa paddhathi thule diy഼echhe thaahale braathy basur mukhe kootaa etaa kathaar kathaa thaahale kootaa niy഼e praakthan mukhyamanthreeke kataaksh
1,prathyek daler prathyek manthree emaelae emapi eder ekataa nijasb kootaa kootaay഼ thaaraa yaake khusi chaakari dithe paaren ebam chirakaal enaaraa sabaai ebhaabe chaakari diy഼eo esechhen೤ baraabarai sambidhaane aajao...
2,bisb jud഼e buddhijeebider sampark thaake bisbabidyaalൟaseminaar paabalisim haaus midiൟaar೤ baamlaay഼ dekhachhi garu chooraakaarabaarider saathe buddhijeebeeraa sampark gad഼e thoolechhe೤abhay഼ paachchhi eraa base arth...
3,bachhar sath sath maanush hathyaa karachhe seemaanthe ganathaanthrik aainer saasaner des bhaarather seemaantharaksheeraa೤ prithibeer koon sabhy deser seemaanthe maanush maanushake guli hathyaa ।iurooper ashtriy഼aa p...
4,samaajer sabatheke dooshith maanush eraai khaaraap dey഼ samaajake chhi chhi chhi
5,yaaraa preranaar kathaar maane bujhhathe paareni thaader maane boojhhaar jany or yethe habe thaandaa masthisk niy഼e bhaaloo boojhhaathe paarabe aamaar dhaaranaa abasyai thaaraa susth maanashikathaar hay഼
6,kandam didi grinning face sweat grinning face sweat
7,bad഼ bad഼ kathaa koothaay഼
8,pade aachhis paaris kar
9,apadaarth chaakaritaa thoor daler nay഼ sarakaari ।bethanabhaathaa sarakaarer thahabil dey഼aa hay഼athoor choorer daler thahabil nay഼ ।


In [ ]:
#To split train and test
X_train1, X_test1, y_train1, y_test1 = train_test_split(df.texttranslit, df1.task_1, test_size=0.25, random_state=42)

In [ ]:
X_train1.shape, X_test1.shape

((960,), (321,))

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='char', ngram_range=(1,3))
tfidf_vect.fit(X_train1)
X_train_tfidf = tfidf_vect.transform(X_train1)
X_test_tfidf = tfidf_vect.transform(X_test1)

In [ ]:
from sklearn.svm import SVC
svm = SVC()

svm .fit(X_train_tfidf, y_train1)

y_pred = svm.predict(X_test_tfidf)

accuracy = accuracy_score(y_test1, y_pred)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test1, y_pred))

Test Accuracy: 63.5514

               precision    recall  f1-score   support

         HOF       0.67      0.21      0.32       131
         NOT       0.63      0.93      0.75       190

    accuracy                           0.64       321
   macro avg       0.65      0.57      0.54       321
weighted avg       0.65      0.64      0.58       321



In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

clf1 = LogisticRegression(random_state=1)
clf2 = BernoulliNB()
clf3 = LinearSVC(penalty='l2',C=1.0)

eclf = VotingClassifier(estimators=[('LR', clf1), ('BNB', clf2),
('SVC', clf3)],  voting='hard')
eclf = eclf.fit(X_train_tfidf, y_train1)
y_pred1 = eclf.predict(X_test_tfidf)
# print(y_pred01)
accuracy3 = accuracy_score(y_test1, y_pred1)

print("Test Accuracy:", round(accuracy3*100, 4))

print("\n", classification_report(y_test1, y_pred1))

Test Accuracy: 62.6168

               precision    recall  f1-score   support

         HOF       0.57      0.35      0.43       131
         NOT       0.65      0.82      0.72       190

    accuracy                           0.63       321
   macro avg       0.61      0.58      0.58       321
weighted avg       0.61      0.63      0.60       321



**Evaluation for test set**

In [ ]:
ldata_1=df2.clean_text
ldata_1=ldata_1.tolist()

In [ ]:
len(ldata_1)

320

In [ ]:
fin=[]
for i in ldata_1:
  t1= getInstance()
  t_text1 = t1.transliterate(str(i), "en_IN")
  fin.append(t_text1)
len(fin)

320

In [ ]:
df_1=pd.DataFrame(fin)
df_1=df_1.rename(columns={0:'texttranslit'})
df_1.head(20)

,texttranslit
0,thaathe thoor kee chulakuni naare saalaa
1,chati chaataa habe
2,kkr prasangg thulale saaharukher chhabi deൟaa baadhyathaamoolak saaharukh kkr sahamaalik paare kriketaar nൟ೤ kriket ebam kriketaarake bhaalabese aapaamar janathaa khelaa dekhe daler maalikake dekhe nൟ೤
3,hyaa hyaa bhikhiri balathei paaser deser looker laagachhe೤ yaar naam dhare daakalei chinathe paare೤
4,etaa nijer meൟer naam raakhaben naam dhare daakaben beaming face smiling eyes grinning face sweat grinning squinting face
5,pigi chapas maane baamlaay഼ oitaai boojhhaay഼ face tears joy face tears joy face tears joy face tears joy face tears joy aamaar mey഼e hale naam raakhab laket face tears joy face tears joy face tears joy face tears jo...
6,maachhele dujanakei bhaaloo laagachhe೤
7,iൟe maathaaൟ otheni೤ eguloo sabai parikalpanaa bidbesh chhaa൜aanoo
8,haaraami balun
9,khabar prachaar karun bhaal kathaa khey഼aal raakhaben yesab maanush asleel bhaashaa byabahaar thaaderake aamanthran karaben


In [ ]:
X_train=df['texttranslit']
X_test=df_1['texttranslit']
y_train=df1['task_1']

In [ ]:
tfidf_vect1 = TfidfVectorizer(analyzer='char', ngram_range=(1,3))
tfidf_vect1.fit(X_train)
X_train_tfidf1 = tfidf_vect1.transform(X_train)
X_test_tfidf1= tfidf_vect1.transform(X_test)

SVM

In [ ]:
from sklearn.svm import SVC
svm1 = SVC(kernel='linear',class_weight='balanced')

svm1 .fit(X_train_tfidf1, y_train)

y_pred2 = svm1.predict(X_test_tfidf1)

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df11= pd.DataFrame(data=y_pred2, columns=['y_pred2'])
Submisssion_bengali = pd.DataFrame()
Submisssion_bengali['S. No.'] = df2['S. No.']
Submisssion_bengali['task_1 '] = y_pred_df11
Submisssion_bengali.to_csv('/content/drive/MyDrive/MUCS_test_Bengali_run18.csv',index = None)

Ensembler

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

clf1 = LogisticRegression(random_state=1)
clf2 = BernoulliNB()
clf3 = LinearSVC(penalty='l2',C=1.0)

eclf1 = VotingClassifier(estimators=[('LR', clf1), ('BNB', clf2),
('SVC', clf3)],  voting='hard')
eclf1 = eclf1.fit(X_train_tfidf1, y_train)
y_pred3 = eclf1.predict(X_test_tfidf1)

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df12= pd.DataFrame(data=y_pred3, columns=['y_pred3'])
Submisssion_bengali = pd.DataFrame()
Submisssion_bengali['S. No.'] = df2['S. No.']
Submisssion_bengali['task_1 '] = y_pred_df12
Submisssion_bengali.to_csv('/content/drive/MyDrive/MUCS_test_Bengali_run19.csv',index = None)